In [36]:
import pandas as pd
import numpy as np
df = pd.read_csv('SFJ6BQ1G.csv')
df=df.set_index(df.columns[0])
df=df.dropna()
df.head
# question3
arith_ret=df.pct_change().dropna()
log_ret=np.log(df/df.shift(1)).dropna()
expected_value=log_ret.mean()
mu_log_daily   = log_ret.mean()
display(arith_ret.head())
display(log_ret.head())
display(expected_value)
cov_matrix = log_ret.cov()
cov_arith_daily = arith_ret.cov()
print("Covariance Matrix of Log Returns:")
display(cov_matrix)
#question 4
#question a
Sigma = cov_matrix
n=len(Sigma.columns)
ones=np.ones((n,1))
Sigma_mat = Sigma.values
try:
    invSigma=np.linalg.inv(Sigma_mat)
except np.linalg.LinAlgError:
    invSigma = np.linalg.pinv(Sigma_mat)  
w_gmv_num = invSigma @ ones
w_gmv_den = (ones.T @ invSigma @ ones).item()
w_gmv = w_gmv_num / w_gmv_den
W_gmv = pd.DataFrame(w_gmv, index=Sigma.columns, columns=['GMV Weight'])
display(W_gmv)
#question b
mu_vec = mu_log_daily.values.reshape(-1, 1) 
rf_daily = 0.02 / 252.0 
excess=mu_vec-rf_daily*ones
num = invSigma @ excess
den=(ones.T@invSigma@excess).item()
w_tan=num / den  
W_tan = pd.DataFrame(w_tan, index=log_ret.columns.tolist(), columns=["Market/Tangency (rf=2%)"])
display(W_tan)
#question c
num_div = invSigma @ mu_vec
den_div = ones.T @ invSigma @ mu_vec
w_div  = pd.DataFrame(w_div, index=log_ret.columns.tolist(), columns=["Diversified (r=0)"])
display(w_div)
#question 5
back_test=60
log_ret_recent=log_ret.tail(back_test)
ret_gmv = log_ret_recent @ w_gmv
ret_tan = log_ret_recent @ w_tan
ret_div = log_ret_recent @ w_div
r_f = 0.02 
TRADING_DAYS = 252
def portfolio_metrics(ret_series, r=r_f):
    mu_daily=ret_series.mean()
    sigma_daily=ret_series.std()
    mu_annual = mu_daily * TRADING_DAYS
    sigma_annual = sigma_daily * np.sqrt(TRADING_DAYS)
    sharpe = (mu_annual - r) / sigma_annual
    return mu_annual, sigma_annual, sharpe
results = pd.DataFrame({
    "Portfolio": ["GMV", "Tangency (r=2%)", "Diversified (r=0)"],
    "Annualized Return": [portfolio_metrics(ret_gmv)[0],
                          portfolio_metrics(ret_tan)[0],
                          portfolio_metrics(ret_div, r=0)[0]],
    "Annualized Volatility": [portfolio_metrics(ret_gmv)[1],
                              portfolio_metrics(ret_tan)[1],
                              portfolio_metrics(ret_div, r=0)[1]],
    "Sharpe Ratio": [portfolio_metrics(ret_gmv)[2],
                     portfolio_metrics(ret_tan)[2],
                     portfolio_metrics(ret_div, r=0)[2]]
})
display(results.round(4))

,AMD,NVDA,PLTR,TSLA,^GSPC
Idx,,,,,
2,0.000410,0.022414,-0.027993,-0.036988,-0.009586
3,0.010704,0.040479,0.065827,0.015239,0.009689
4,-0.010301,-0.001806,0.040531,-0.014110,0.007113
5,-0.039995,0.016284,0.009042,-0.009459,-0.002070
6,0.022597,-0.000074,-0.000230,-0.087825,0.006052


,AMD,NVDA,PLTR,TSLA,^GSPC
Idx,,,,,
2,0.000410,0.022167,-0.028392,-0.037690,-0.009632
3,0.010647,0.039681,0.063751,0.015124,0.009643
4,-0.010354,-0.001808,0.039731,-0.014211,0.007088
5,-0.040817,0.016152,0.009002,-0.009504,-0.002072
6,0.022346,-0.000074,-0.000230,-0.091924,0.006034


AMD     -0.000149
NVDA     0.001635
PLTR     0.005882
TSLA     0.002175
^GSPC    0.000623
dtype: float64

Covariance Matrix of Log Returns:


,AMD,NVDA,PLTR,TSLA,^GSPC
AMD,0.001017,0.000641,0.000650,0.000642,0.000261
NVDA,0.000641,0.000982,0.000698,0.000596,0.000264
PLTR,0.000650,0.000698,0.001964,0.000931,0.000291
TSLA,0.000642,0.000596,0.000931,0.001947,0.000326
^GSPC,0.000261,0.000264,0.000291,0.000326,0.000138


,GMV Weight
AMD,-0.101355
NVDA,-0.132642
PLTR,-0.032853
TSLA,-0.094509
^GSPC,1.361359


,Market/Tangency (rf=2%)
AMD,-1.711385
NVDA,0.519888
PLTR,1.825765
TSLA,0.022751
^GSPC,0.342982


,Diversified (r=0)
AMD,-1.194749
NVDA,0.310500
PLTR,1.229361
TSLA,-0.014876
^GSPC,0.669764


,Portfolio,Annualized Return,Annualized Volatility,Sharpe Ratio
0,GMV,0 0.095205 dtype: float64,0 0.094161 dtype: float64,0 0.798679 dtype: float64
1,Tangency (r=2%),0 0.954629 dtype: float64,0 0.876672 dtype: float64,0 1.06611 dtype: float64
2,Diversified (r=0),Diversified (r=0) 0.678852 dtype: float64,Diversified (r=0) 0.602883 dtype: float64,Diversified (r=0) 1.12601 dtype: float64
